In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Models.Decoder import Decoder
from Models.Encoder import Encoder
from utils.converter import str2ncs, ncs2str
from Models.Decoder.Decoder import DFS, BFS, Beam
import time
import numpy as np


In [10]:
# 1k1050 10k450 1k0350 1k03100 1k01100
en = Encoder()
origins, reads = en.load_data(path='data/20010100')
origins.shape, reads.shape

((200,), (200, 100))

In [ ]:
init_kmer_size = 8
prune = 6

kmer_sizes = []
accuracies = []

for kmer_size in range(init_kmer_size, 100, 4):
    kmer_sizes.append(kmer_size)
    # kmer_size = 50
    total_time = 0
    bfs_results = []
    print(f'kmer_size: {kmer_size} prune: {prune}')
    for i, original in enumerate(origins):
        decoder = Decoder(origin=original, prune=prune)
        decoder.build_graph(reads[i], len(original), kmer_size=kmer_size, visualization=False)
        start = time.time()
        rs = decoder.decode(BFS(min_weight=prune, db=10))
        ttt = time.time() - start
        total_time += round(ttt, 3)
        bfs_results.append(rs[0])
        if i%20 == 0:
            print(f'{i}: {round(ttt, 3)}s {round(total_time, 3)}s {round(np.array(bfs_results).sum()*100/(i+1), 4)}%')
    accuracies.append(round(np.array(bfs_results).sum()*100/original.shape[0], 4))


In [17]:
init_k = 5
prune = 6
kmer_size = 24
k_sizes = []
accuracies = []

for k in range(init_k, 100, 5):
    k_sizes.append(k)
    # kmer_size = 50
    total_time = 0
    bfs_results = []
    print(f'kmer_size: {kmer_size} k: {k}')
    for i, original in enumerate(origins):
        decoder = Decoder(origin=original, prune=prune)
        decoder.build_graph(reads[i], len(original), kmer_size=kmer_size, visualization=False)
        start = time.time()
        rs = decoder.decode(Beam(k=k, db=0))
        ttt = time.time() - start
        total_time += round(ttt, 3)
        bfs_results.append(rs[0])
        if i%20 == 0:
            print(f'{i}: {round(ttt, 3)}s {round(total_time, 3)}s {round(np.array(bfs_results).sum()*100/(i+1), 4)}%')
    accuracies.append(round(np.array(bfs_results).sum()*100/original.shape[0], 4))


kmer_size: 24 k: 5
0: 0.011s 0.011s 100.0%
20: 0.006s 0.19s 47.619%
40: 0.009s 0.37s 46.3415%
60: 0.003s 0.525s 42.623%
80: 0.011s 0.704s 43.2099%
100: 0.011s 0.882s 45.5446%
120: 0.007s 1.047s 44.6281%
140: 0.009s 1.23s 47.5177%
160: 0.01s 1.397s 48.4472%
180: 0.01s 1.57s 49.1713%


KeyboardInterrupt: 